In [6]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload complete

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import matplotlib.pyplot as plt
import xarray as xr

from project.test_pendulum import *
from project.util import *

In [8]:
def make_dataset():
    tt, xx, yy = run_oscillator(t_end=200)

    ds = xr.Dataset({
        "x": (["time"], yy[0,:]),
        "v": (["time"], yy[1,:]),
    }, coords={
        "time": tt
    })

    return ds

ds = make_dataset()
ds

<xarray.Dataset>
Dimensions:  (time: 2000)
Coordinates:
  * time     (time) float64 0.0 0.1 0.2 0.3 0.4 ... 199.6 199.7 199.8 199.9
Data variables:
    x        (time) float64 0.9534 0.9617 0.9904 ... -0.01613 0.02348 -0.03679
    v        (time) float64 -0.03936 -0.1436 0.03278 ... -0.00617 -0.07552

In [9]:
data = stack_state(ds)
data

<xarray.DataArray '' (elem: 2, time: 2000)>
array([[ 0.95337825,  0.96165724,  0.99044209, ..., -0.01613219,
         0.02348349, -0.03678628],
       [-0.03935893, -0.14358031,  0.03278479, ..., -0.07462803,
        -0.00617032, -0.07552383]])
Coordinates:
  * time     (time) float64 0.0 0.1 0.2 0.3 0.4 ... 199.6 199.7 199.8 199.9
  * elem     (elem) object MultiIndex
  * field    (elem) object 'x' 'v'

In [10]:
taus = data["time"].diff("time").values
assert all(np.isclose(taus[0], taus))  # must have uniform dt
tau0 = taus[0].item()
tau0

0.1

In [50]:
data_t0 = data.isel(time=slice(None, -1))
data_t0

<xarray.DataArray '' (elem: 2, time: 1999)>
array([[ 0.95337825,  0.96165724,  0.99044209, ...,  0.0210218 ,
        -0.01613219,  0.02348349],
       [-0.03935893, -0.14358031,  0.03278479, ..., -0.07591145,
        -0.07462803, -0.00617032]])
Coordinates:
  * time     (time) float64 0.0 0.1 0.2 0.3 0.4 ... 199.5 199.6 199.7 199.8
  * elem     (elem) object MultiIndex
  * field    (elem) object 'x' 'v'

In [12]:
data_tau0 = data.isel(time=slice(1, None))
data_tau0

<xarray.DataArray '' (elem: 2, time: 1999)>
array([[ 0.96165724,  0.99044209,  0.93772801, ..., -0.01613219,
         0.02348349, -0.03678628],
       [-0.14358031,  0.03278479,  0.01827475, ..., -0.07462803,
        -0.00617032, -0.07552383]])
Coordinates:
  * time     (time) float64 0.1 0.2 0.3 0.4 0.5 ... 199.6 199.7 199.8 199.9
  * elem     (elem) object MultiIndex
  * field    (elem) object 'x' 'v'

In [49]:
C_tau0 = estimate_cov(data_tau0, data_t0)
C_t0 = estimate_cov(data_t0, data_t0)
G = xr.dot(C_tau0, inverse(C_t0), dims=[])
G

<xarray.DataArray (elem1: 2, elem2: 2)>
array([[ 1.00000106, -0.00311992],
       [-0.00311992,  1.00361329]])
Coordinates:
  * elem1    (elem1) object MultiIndex
  * field1   (elem1) <U1 'x' 'v'
  * elem2    (elem2) object MultiIndex
  * field2   (elem2) <U1 'x' 'v'